класификатор на основе предобученной ResNet101 с функцией потерь ArcFace. 
Обучен на части VGG Face (1016 класов).
Слой ембендингов лица - 512 параметров

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import cv2

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import *
import keras.backend as K
from keras.models import Sequential

In [10]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.
RANDOM_SEED          = 100

EPOCHS               = 10  # эпох на обучение
BATCH_SIZE           = 64 # 
LR                   = 0.0001
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 1016 
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

#PATH = "../input/dog-vs-cat/dvc/" # рабочая директория
#PATH = "../input/modified-vggface2/train_refined_resized/" # рабочая директория
PATH = "../input/vgg1016/data3/"

In [11]:
#Аугментация данных
train_datagen = ImageDataGenerator(
    rescale=1. / 255, #коэффициент масштабирования. По умолчанию - Нет. Если Нет или 0, 
    #масштабирование не применяется, в противном случае мы умножаем данные на предоставленное значение 
    #(после применения всех остальных преобразований).
    
    rotation_range = 5, #угол случайных поворотов
    width_shift_range = 0.1, #диапазон сдвига ширины
    height_shift_range = 0.1, #диапазон сдвига высоты
    #brightness_range =  [0.5, 1.5], # сдвиг яркости
    validation_split = VAL_SPLIT, # Доля выборки для валидации 
    horizontal_flip = True  # отражение по горизонтали
    )

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [12]:
#Генерация данных
train_generator = train_datagen.flow_from_directory(
    PATH,#+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH,#+'train/',    
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

# **категориальная кроссэнтропия**

![](https://i.stack.imgur.com/LSVEc.png)

In [14]:
#категориальная кроссэнтропия
import keras.backend as K
from tensorflow.python.ops import math_ops

def my_c_crossentropy(y_true, y_pred):
    r = -K.sum(y_true * K.log(y_pred), axis=1) # сумма по классам
    return r

# **softmax + категориальная кроссэнтропия = softmax_loss**

![](https://lh3.googleusercontent.com/wC2QldxOFZK0CXcOKo9nApAM6FWvjtgJQChzG6Unxh31WPGFF9BqPEKczg_6rHGkb_to_HXG7ko-OH35nfFu4JhgPmp6Ycmyfs9dr78ohdLKt7TLvTWvUoOBpaMPQu3J3firAAMh)

In [15]:
def softmax_loss(y_true, W, b, x):
    
    y_pred = tf.matmul(x, W) + b
    numerators = tf.reduce_sum(y_true * tf.exp(y_pred), axis=1)
    denominators = tf.reduce_sum(tf.exp(y_pred), axis=1)
    loss = - tf.reduce_sum(K.log(numerators / denominators))

    return loss

# **Center Loss**

![](https://lh6.googleusercontent.com/PzzKwazqd6uLdmIBrCFLxeDBIlVsUazP8Xm-7Q-USkJboqlzEhs5ml6g0HY3FbV_Z35VnRMvYnL6XGZwm_BYkzZAmO-Mi2sWOpt-Llvh94y7DvnV8v_KHD_3K55FOKxicUs9QFjX)

In [16]:
import tensorflow as tf

def circle_loss(W, b, lamda_center):
    
    def inner(y_true, x):
        y_pred = tf.matmul(x, W) + b
        numerators = tf.reduce_sum(y_true * tf.exp(y_pred), axis=1)
        denominators = tf.reduce_sum(tf.exp(y_pred), axis=1)
        loss_softmax = - tf.reduce_sum(K.log(numerators / denominators))

        class_freqs = tf.reduce_sum(y_true, axis=0, keepdims=True)
        class_freqs = tf.transpose(class_freqs)

        centres = tf.matmul(tf.transpose(y_true), x)
        centres = tf.divide(centres, class_freqs)
        repeated_centres = tf.matmul(y_true, centres)

        sq_distances = tf.square(tf.norm(x - repeated_centres, axis=1))
        loss_centre = tf.reduce_sum(sq_distances)

        loss = loss_softmax + (lambda_center/2) * loss_centre 

        return loss
    return inner

# **A-Softmax (aka SphereFace)**

![](https://lh6.googleusercontent.com/SrP3-1kes_eBb6wJV7jVOdymbT5oYjkIJ9-6ABB49zZ0VGyA5aMjrm1AAW3sV7nFZsAIXeOJSN3EYOQAfalGsDwJjQjE3ppDMneUoYl6JKseMYk1DvzQ-adMRRvwrVnTuO4yTGLg)

In [17]:
def SphereFaceLoss(W, x, m):
    def inner(y_true, x):
    
        # replace 0 => 1 and 1=> m in y_true
        M = (m-1) * y_true + 1
    
        # consider normalized weight matrix
        normalized_W, norms = tf.linalg.normalize(W, axis=0)
    
        # get dot products (projections)
        y_pred = x * normalized_W

        # W . x = ||W||*||x||*cos(theta)
        # but ||W|| = 1
        # so (W . x) / ||x|| = cos(theta) 
    
        cos_theta, norm_x = tf.linalg.normalize(y_pred, axis=1)    
        theta = tf.acos(cos_theta)
    
        # multiply theta by appropriate margin
        new_theta = theta * M
        new_cos_theta = tf.cos(new_theta)
        new_y_pred = norm_x * new_cos_theta

        # the following part is the same as softmax loss
        numerators = tf.reduce_sum(y_true * tf.exp(new_y_pred), axis=1)
        denominators = tf.reduce_sum(tf.exp(new_y_pred), axis=1)
        loss = - tf.reduce_sum(K.log(numerators / denominators))

        return loss
    return inner

# **Large Margin Cosine Loss (aka CosFace)**

![](https://lh5.googleusercontent.com/GD1d8wP1WLM3Wr0VMRMmYcdjWLnHpoOg03l4_pbM_v6PSFs4iCW3po5Fblzy5G4NMPJC4SaDcwV6o4Gbe8OZ7aR0feoPnJwyah1iqpUebzUsHkypEXlcSAcRh6AWaw52v86eah5K)

In [18]:
def CosFaceLoss(W, m, s):
    def inner(y_true, x):
        # replace 0 => 1 and 1=> m in y_true
        y_true = tf.cast(y_true, dtype=tf.float32)
        M = m * y_true

        # W . x = ||W|| * ||x|| * cos(theta)
        # so (W . x) / (||W|| * ||x||) = cos(theta)

        dot_product = tf.matmul(x, W)
        cos_theta, cos_theta_norm = tf.linalg.normalize(dot_product,axis=0)

        # re-scale the cosines by a hyper-parameter s
        # and subtract appropriate margin
        y_pred = s * cos_theta - M

        # the following part is the same as softmax loss
        numerators = tf.reduce_sum(y_true * tf.exp(y_pred), axis=1)
        denominators = tf.reduce_sum(tf.exp(y_pred), axis=1)
        loss = - tf.reduce_sum(tf.math.log(numerators/denominators))
        return loss
    return inner

# **Additive Angular Margin Loss (aka ArcFace)**

![](https://lh6.googleusercontent.com/5GX2-zIqIKM-Ge6dqiK9hikijptd_5L5ARYU4BbaG4qtHJHJpDs3lSBBCUOBogSHCFjdXpeffbOR_bypzWAXHFPTn6ThHrYin7Vf1R4e2-WrTgOgY4DtJCcFXgpI6tI8gGYb5AMQ)

In [19]:
def ArcFaceLoss(W, m, s):
    def inner(y_true, x):
        # replace 0 => 1 and 1=> m in y_true
        M = (m-1) * y_true + 1
        
        # W . x = ||W||*||x||*cos(theta)
        # but ||W|| = 1 and ||x|| = 1
        # so (W . x) = cos(theta) 
        dot_product = tf.matmul(x, W)
        cos_theta,cos_theta_norms = tf.linalg.normalize(dot_product,axis=0)

        theta = tf.acos(cos_theta)
    
        # add appropriate margin to theta
        new_theta = theta + M
        new_cos_theta = tf.cos(new_theta)
    
        # re-scale the cosines by a hyper-parameter s
        y_pred = s * new_cos_theta     

        # the following part is the same as softmax loss
        numerators = tf.reduce_sum(y_true * tf.exp(y_pred), axis=1)
        denominators = tf.reduce_sum(tf.exp(y_pred), axis=1)
        loss = - tf.reduce_sum(K.log(numerators / denominators))
    
        return loss

    return inner

In [20]:
#Строим модель
#Загружаем предобученную сеть :
#base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)
base_model = ResNet101(weights='imagenet', include_top=False, input_shape = input_shape)
#base_model.trainable = False
#base_model.summary()
#Устанавливаем новую «голову» (head):
x = base_model.output
x = GlobalAveragePooling2D()(x)  #Pooling слой
x = BatchNormalization()(x) #добавим Batch нормализацию
x = Dense(512, activation='elu')(x) # полносвязный слой с активацией elu
x = Dropout(0.25)(x) # полносвязный слой с вероятность отключения нейронов в слое
predictions = Dense(CLASS_NUM, activation='sigmoid')(x) #логистический слой c 2 классами

predictions.trainable = False

#model = Model(inputs=base_model.input, outputs=predictions)
model = tf.keras.models.Model(inputs=base_model.input, outputs=[x, predictions])


def dummy_loss(ytrue, ypred):
    return tf.constant([0])

#loss_func = CosFaceLoss(W=model.layers[-1].weights[0], m=10.0, s=10.0)
loss_func = ArcFaceLoss(W=model.layers[-1].weights[0], m=0.2, s=10.0)

model.compile(loss=[loss_func, dummy_loss], optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])


In [21]:
base_model.trainable = True

In [22]:
#Добавим ModelCheckpoint. Эта функция позволяет сохранять прогресс обучения модели,
#чтобы в нужный момент можно было его подгрузить и дообучить модель.
checkpoint = ModelCheckpoint('dvc.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')

#останоить обучение, когда метрика перестает улучшаться.
earlystop = EarlyStopping(monitor = 'val_accuracy', patience = 4, restore_best_weights = True)

#Снижайть скорость обучения, когда метрика перестает улучшаться.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2, min_lr=0.0000001, verbose=1, mode='auto')
    
callbacks_list = [checkpoint, earlystop, reduce_lr]

In [23]:
#Обучаем модель
history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
test_img = '../input/test-dvc/test1/1.jpg'

In [ ]:
from matplotlib import pyplot
from matplotlib.image import imread
image = imread(test_img)
# plot raw pixel data
pyplot.imshow(image)
# show the figure
pyplot.show()
image = cv2.imread(test_img)[..., ::-1]
image3 = cv2.resize(image, (224, 224))/255
pred = model.predict(image3[None, ...])
#print(pred)
predict = np.argmax(pred[1], axis=-1)
#print(predictions)
if predict == 0:
    print('it`s cat')
else:
    print('it`s dog') 